Created: 7/1/2025

Purpose: Exploratory Data analysis on the generated model dataset

In [2]:
# import libraries and functions

from heatmap_analysis import Report, gather_model_paths, load_references, open_log, step_file_system
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

This is the setup to load dataset into memory for further analysis

In [29]:
class_path = "D:\\model_dataset\\classification"
regress_path = "D:\\model_dataset\\regression"
model_paths = gather_model_paths(class_path, regress_path)
class_models = load_references(model_paths[1])
regress_models = load_references(model_paths[0])
class_logs_agg = []
regress_logs_agg = []
class_by_dataset = {}
regress_by_dataset = {}

# Gather the data for the classification models
for model in class_models:
    log = open_log(model.model_path)
    class_logs_agg.append(log)
    rows = model.model_path.split('\\')[-2]
    if rows in class_by_dataset:
        class_by_dataset[rows].append(log)
    else:
        class_by_dataset.update({rows: [log]})

#Gather the data fro the regression models
for model in regress_models:
    log = open_log(model.model_path)
    regress_logs_agg.append(log)
    rows = model.model_path.split('\\')[-2]
    if rows in regress_by_dataset:
        regress_by_dataset[rows].append(log)
    else:
        regress_by_dataset.update({rows: [log]})

In [ ]:
class_data = pd.DataFrame(class_by_dataset)
regress_data = pd.DataFrame(regress_by_dataset)
class_acc = class_data.copy().map(lambda x: x.accuracy)
regress_acc = regress_data.copy().map(lambda x: x.accuracy)